In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK070072,Karma Resorts,Other,Past Guests,Karma Kandara,PF304376,Miss,Heidi Lea,Robinson,Female,...,2025-10-19 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
1,KK080199,Karma Resorts,Other,Past Guests,Karma Kandara,PF256274,Mrs,Sherrin Sook Ping,Loh,Female,...,2025-10-21 00:00:00,5,HOUSE_USE,V_OWNER,COMP,2,0,0,Email and Phone,NO
2,KK081385,Karma Resorts,Other,Past Guests,Karma Kandara,PF333250,Miss,Nicky Louise,Batten,Female,...,2025-10-19 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,3,0,Email and Phone,NO
3,KK077721,Karma Resorts,Other,Past Guests,Karma Kandara,PF323352,Mr.,Stuart Edward Campbell,Rogers,Male,...,2025-10-19 00:00:00,2,DIGITAL,WEDDING,GROUP,3,0,0,Email and Phone,NO
4,KK080658,Karma Resorts,Other,Past Guests,Karma Kandara,PF329490,Mrs,Kristin,De Witte,Female,...,2025-10-21 00:00:00,21,COMP,STAFF,COMP,1,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,KB053891,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332285,Mr,Robert,Schade,Male,...,2025-10-24 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
68,KB053887,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332285,Mr,Robert,Schade,Male,...,2025-10-24 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
69,KB053889,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332285,Mr,Robert,Schade,Male,...,2025-10-20 00:00:00,1,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
70,KB053890,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332285,Mr,Robert,Schade,Male,...,2025-10-21 00:00:00,1,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
14,KB051203,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333614,Herr,A.,Fügen,Male,...,2025-10-19 00:00:00,2,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
22,KB053171,Karma Resorts,Other,Past Guests,Karma Bavaria,PF018070,Mr,Rolf,Badenhoop,Male,...,2025-10-23 18:30:00,0,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
23,KB053170,Karma Resorts,Other,Past Guests,Karma Bavaria,PF018070,Mr,Rolf,Badenhoop,Male,...,2025-10-23 23:00:00,0,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
25,KB053166,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329227,Dr,Simon,Brosseder,Male,...,2025-10-23 23:00:00,0,DIR,TEL,RETAIL,1,0,0,Email Only,NO
26,KB053167,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329227,Dr,Simon,Brosseder,Male,...,2025-10-23 23:00:00,0,DIR,TEL,RETAIL,1,0,0,Email Only,NO
27,KB053177,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329227,Dr,Simon,Brosseder,Male,...,2025-10-23 23:00:00,0,DIR,TEL,RETAIL,1,0,0,Email Only,NO
28,KB053165,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329227,Dr,Simon,Brosseder,Male,...,2025-10-23 22:30:00,0,DIR,TEL,RETAIL,1,0,0,Email Only,NO
34,KK081740,Karma Resorts,Other,Past Guests,Karma Kandara,PF332617,Mr,Masood,Khan,Male,...,2025-10-26 00:00:00,1,COMP,STAFF,COMP,5,0,0,Email and Phone,NO
37,KK081632,Karma Resorts,Other,Past Guests,Karma Kandara,PF329292,Mr,Dimas Trisetyo,Nugroho,Male,...,2025-10-22 00:00:00,1,DIR,CHAT,DIGITAL,2,0,0,Email and Phone,NO
46,KB054044,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332634,Ms,Eva-Maria,Boerschlein,Unknown,...,2025-10-24 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email Only,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK070072,Karma Resorts,Other,Past Guests,Karma Kandara,PF304376,Miss,Heidi Lea,Robinson,Female,...,2025-10-19 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
1,KK080199,Karma Resorts,Other,Past Guests,Karma Kandara,PF256274,Mrs,Sherrin Sook Ping,Loh,Female,...,2025-10-21 00:00:00,5,HOUSE_USE,V_OWNER,COMP,2,0,0,Email and Phone,NO
2,KK081385,Karma Resorts,Other,Past Guests,Karma Kandara,PF333250,Miss,Nicky Louise,Batten,Female,...,2025-10-19 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,3,0,Email and Phone,NO
3,KK077721,Karma Resorts,Other,Past Guests,Karma Kandara,PF323352,Mr.,Stuart Edward Campbell,Rogers,Male,...,2025-10-19 00:00:00,2,DIGITAL,WEDDING,GROUP,3,0,0,Email and Phone,NO
4,KK080658,Karma Resorts,Other,Past Guests,Karma Kandara,PF329490,Mrs,Kristin,De Witte,Female,...,2025-10-21 00:00:00,21,COMP,STAFF,COMP,1,0,0,Email and Phone,NO
5,KK076553,Karma Resorts,Other,Past Guests,Karma Kandara,PF319989,Mr.,Etienne Remi Christophe,Daury,Male,...,2025-10-20 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
6,KK073527,Karma Resorts,Other,Past Guests,Karma Kandara,PF311969,Mr,Pascal,Oudin,Male,...,2025-10-19 00:00:00,2,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
7,KB050050,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333788,Frau,Anja,Krieger,Female,...,2025-10-19 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
8,KB053738,Karma Resorts,Other,Past Guests,Karma Bavaria,PF331526,Frau,Brigitte,Baierlein,Female,...,2025-10-20 18:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
9,KB050826,Karma Resorts,Other,Past Guests,Karma Bavaria,PF072877,Herr,Dieter,Nees,Male,...,2025-10-20 00:00:00,14,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
9,KB050826,Karma Resorts,Other,Past Guests,Karma Bavaria,PF072877,Herr,Dieter,Nees,Male,...,2025-10-20 00:00:00,14,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
12,KB051204,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333611,Herr,A.,Sebald,Male,...,2025-10-19 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
13,KB051202,Karma Resorts,Other,Past Guests,Karma Bavaria,PF250879,Herr,Georg,Freundorfer,Male,...,2025-10-19 00:00:00,2,COMP,V_OWNER,COMP,1,0,0,Email and Phone,NO
14,KB045417,Karma Resorts,Other,Past Guests,Karma Bavaria,PF280810,Herr,Harald,Brixner,Female,...,2025-10-24 00:00:00,4,COMP,GER_OWE,COMP,3,0,0,Email and Phone,NO
15,KB045441,Karma Resorts,Other,Past Guests,Karma Bavaria,PF323967,empty,Joachim,Schug,Unknown,...,2025-10-24 00:00:00,4,COMP,GER_OWE,COMP,3,0,0,Email and Phone,NO
20,KB053168,Karma Resorts,Other,Past Guests,Karma Bavaria,PF018070,Mr,Rolf,Badenhoop,Male,...,2025-10-23 23:00:00,0,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
36,KB053958,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332634,Ms,Eva-Maria,Boerschlein,Unknown,...,2025-10-22 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
37,KK072933,Karma Resorts,Other,Past Guests,Karma Kandara,PF267746,Mr,Chirstoph,Fischer,Male,...,2025-10-21 00:00:00,26,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK070072,Karma Resorts,Other,Past Guests,Karma Kandara,PF304376,Miss,Heidi Lea,Robinson,Female,...,2025-10-19 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
1,KSM015583,Karma Resorts,Other,Past Guests,Karma St Martins,PF314483,Miss,Saski,Bridges,Female,...,2025-10-19 00:00:00,1,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
2,KSM017273,Karma Resorts,Other,Past Guests,Karma St Martins,PF156225,Mr.,Phillip,Roberts,Male,...,2025-10-19 00:00:00,3,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
3,KSM014678,Karma Resorts,Other,Past Guests,Karma St Martins,PF301497,Miss,Charlotte,Marsh,Unknown,...,2025-10-19 00:00:00,3,COMP,COMPETITION,MEDIA,2,0,0,Email and Phone,NO
4,KK080881,Karma Resorts,Other,Past Guests,Karma Kandara,PF332064,Mr.,Abdulrahman Omar,Bin Otaysh,Male,...,2025-10-19 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
5,KK074355,Karma Resorts,Other,Past Guests,Karma Kandara,PF314194,Ms,Raissa,Lauwsen,Female,...,2025-10-19 00:00:00,2,DIGITAL,WEDDING,GROUP,3,0,0,Email and Phone,NO
6,KK081514,Karma Resorts,Other,Past Guests,Karma Kandara,PF333570,Ms,Maizura,-,Female,...,2025-10-19 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KK077867,Karma Resorts,Other,Past Guests,Karma Kandara,PF141616,Ms,Lanny,Wijaya,Female,...,2025-10-19 00:00:00,1,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
8,KK081383,Karma Resorts,Other,Past Guests,Karma Kandara,PF333247,Mr,.,Vikram,Male,...,2025-10-19 00:00:00,1,HOUSE_USE,GSTAFF,COMP,1,0,0,Email and Phone,NO
9,KB050050,Karma Resorts,Other,Past Guests,Karma Bavaria,PF311759,empty,Stefanie,Hasler,Unknown,...,2025-10-19 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Phone Only,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_8608\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK070072,Karma Resorts,Other,Past Guests,Karma Kandara,PF304376,Miss,Heidi Lea,Robinson,Female,...,2025-10-19,3,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
1,KB050050,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333788,Frau,Anja,Krieger,Female,...,2025-10-19,2,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
2,KK073527,Karma Resorts,Other,Past Guests,Karma Kandara,PF311969,Mr,Pascal,Oudin,Male,...,2025-10-19,2,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
3,KB053905,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332327,Herr,Franz,Fellner,Male,...,2025-10-19,1,FF,TEL,RETAIL,2,0,0,Email and Phone,NO
4,KB054086,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333220,Frau,Leonie,Sixt,Unknown,...,2025-10-19,1,OTA,OTA,RETAIL,5,1,0,Email and Phone,NO
5,KK077721,Karma Resorts,Other,Past Guests,Karma Kandara,PF323352,Mr.,Stuart Edward Campbell,Rogers,Male,...,2025-10-19,2,DIGITAL,WEDDING,GROUP,3,0,0,Email and Phone,NO
6,KK081385,Karma Resorts,Other,Past Guests,Karma Kandara,PF333250,Miss,Nicky Louise,Batten,Female,...,2025-10-19,3,HOUSE_USE,V_OWNER_NRB,COMP,2,3,0,Email and Phone,NO
7,KK073425,Karma Resorts,Other,Past Guests,Karma Kandara,PF311604,Mr,.,Dede,Male,...,2025-10-19,1,HOUSE_USE,WEDDING,GROUP,1,0,0,Email and Phone,NO
8,KK081564,Karma Resorts,Other,Past Guests,Karma Kandara,PF333513,Mrs,Nicola Patricia,Peters,Female,...,2025-10-19,1,DIR,CHAT,DIGITAL,2,0,0,Email and Phone,NO
9,KB053287,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328993,,Anastasia,Gazou,Unknown,...,2025-10-19,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 72
Jumlah Data Yang Duplicate : 17
Jumlah Data Setelah Hapus Duplicate : 55
Jumlah Data Yang Termasuk Member: 8
Jumlah Data Setelah Hapus Member : 47
Jumlah Data Akhir : 47


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)